# <center> CS559 Homework#3: Decision Tree and Ensemble Methods</center>
## <center> Due: 11/8/2021 Monday at 11:59 PM</center>


In this assignment, you are going to implement four classifiers - **decision tree, random forest, adaboost, and gradient boost**. 
Then check the performance with `sklearn` built-in algorithms.
In this work, splitting into train and test sets is not necessary. 

The provided data has four columns - three features (a, b, and c) and the target (class). Three features are continuous data and the target is a binary, 0 or 1. 

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings("ignore")

In [45]:
df = pd.read_csv('./F21_CS559_HW3_data.csv')


In [46]:
df.head(5)

,a,b,c,class
0,9.4202,-4.3507,10.3764,1
1,9.7044,-4.4601,10.6803,1
2,9.8075,-4.0894,10.6259,1
3,9.2771,-4.0349,10.1166,1
4,9.6447,-3.5968,10.2936,1


### Question 1: Decisition Tree Classifier
- A simple DT implementation (10 pts.)
    - to make the problem simple, implement a decision tree with depth of 3 (the root index is 0).
    - calculate the gini index for each attribute and pick the best attribute for each node.
    - calculate the accuracy using accuracy score. 
- Classification using DecistionTreeClassifier (5 pts)
- Evaluation (5 pts)

In [47]:
def DecisionTree(df):
    X = df.drop('class', 1)
    y = df['class']
    for x in df.columns[0:-1]:
        #Cut by mean
        mean = df[x].mean()
        min = df[x].min()
        max = df[x].max()
        mid = (max + min) /2

        df[x] = pd.cut(df[x], bins=[min, mid, max], labels=[0,1])
    def calcGini(item):
    #Calculate gini
        count = Counter(item)
        total = len(item)
        count_list = [count[0], count[1]]
        freqs = [float(x)/total for x in count_list]
        gini_scores = [x*(1-x) for x in freqs]
        final_score = sum(gini_scores)
        return final_score

    def getSortedFeatureScores(df):
        scores = {}
        for feat in df.columns[0:-1]:
            L=list(df[feat])
            scores[feat] = calcGini(L)
        return sorted(scores.items(), key=lambda x: x[1], reverse=True)

    feat_order = []
    pred = {}

    node = getSortedFeatureScores(df)
    feat_order.append(node[0][0])
    l_df = df.loc[df[node[0][0]] == 0]
    r_df = df.loc[df[node[0][0]] == 1]
    #       c
    #     0   1
    #   a       a
    #  0 1     0 1
    # b   b    b  b
    #0 1 0 1  0 1 0 1
    #2 2 2 2  1 1 1 2
    #process left
    n_df = l_df.drop(node[0][0],1)
    node2 = getSortedFeatureScores(n_df)
    ll_df = n_df.loc[n_df[node2[0][0]] == 0]
    ll_df = ll_df.drop(node2[0][0],1)
    rr_df = n_df.loc[n_df[node2[0][0]] == 1]
    rr_df = rr_df.drop(node2[0][0],1)
    # percentage of b1 -> class 1 vs class 2
    total = len(ll_df)
    b0 = ll_df[ll_df[node2[1][0]] ==0]
    pc1 = len(b0[b0['class'] == 1]) / len(b0)
    pc2 = len(b0[b0['class'] == 2]) / len(b0)
    if pc1 > pc2:
        pred['000'] = 1
    else:
        pred['000'] = 2

    b1 = ll_df[ll_df[node2[1][0]] ==1]
    pc1 = len(b1[b1['class'] == 1]) / len(b1)
    pc2 = len(b1[b1['class'] == 2]) / len(b1)
    if pc1 > pc2:
        pred['001'] = 1
    else:
        pred['001'] = 2

    b0 = rr_df[rr_df[node2[1][0]] ==0]
    pc1 = len(b0[b0['class'] == 1]) / len(b0)
    pc2 = len(b0[b0['class'] == 2]) / len(b0)
    if pc1 > pc2:
        pred['010'] = 1
    else:
        pred['010'] = 2

    b1 = rr_df[rr_df[node2[1][0]] ==1]
    pc1 = len(b1[b1['class'] == 1]) / len(b1)
    pc2 = len(b1[b1['class'] == 2]) / len(b1)
    if pc1 > pc2:
        pred['011'] = 1
    else:
        pred['011'] = 2


    #process right
    n_df = r_df.drop(node[0][0],1)
    node2 = getSortedFeatureScores(n_df)
    ll_df = n_df.loc[n_df[node2[0][0]] == 0]
    ll_df = ll_df.drop(node2[0][0],1)
    rr_df = n_df.loc[n_df[node2[0][0]] == 1]
    rr_df = rr_df.drop(node2[0][0],1)
    # percentage of b1 -> class 1 vs class 2
    total = len(ll_df)
    b0 = ll_df[ll_df[node2[1][0]] ==0]
    pc1 = len(b0[b0['class'] == 1]) / len(b0)
    pc2 = len(b0[b0['class'] == 2]) / len(b0)
    if pc1 > pc2:
        pred['100'] = 1
    else:
        pred['100'] = 2

    b1 = ll_df[ll_df[node2[1][0]] ==1]
    pc1 = len(b1[b1['class'] == 1]) / len(b1)
    pc2 = len(b1[b1['class'] == 2]) / len(b1)
    if pc1 > pc2:
        pred['101'] = 1
    else:
        pred['101'] = 2

    b0 = rr_df[rr_df[node2[1][0]] ==0]
    pc1 = len(b0[b0['class'] == 1]) / len(b0)
    pc2 = len(b0[b0['class'] == 2]) / len(b0)
    if pc1 > pc2:
        pred['110'] = 1
    else:
        pred['110'] = 2

    b1 = rr_df[rr_df[node2[1][0]] ==1]
    pc1 = len(b1[b1['class'] == 1]) / len(b1)
    pc2 = len(b1[b1['class'] == 2]) / len(b1)
    if pc1 > pc2:
        pred['111'] = 1
    else:
        pred['111'] = 2

    pred_test = []
    #do predictions
    for index, row in df.iterrows():
        key = str(row['c']) + str(row['a']) + str(row['b'])
        try:
            pred_test.append(pred[key])
        except:
            pred_test.append(1)
    score = accuracy_score(pred_test, y)
    return score
selfscore = DecisionTree(df)

In [48]:
dtc = DecisionTreeClassifier()
dtc.fit(X,y)
score = dtc.score(X,y)
score

1.0

In [49]:
print(f"Implement score: {selfscore}")
print(f"sklearn score: {score}")

Implement score: 0.972
sklearn score: 1.0


### Question 2: Random Forest Classifier
- A simle RF implementation (10 pts)
    - make a bootstrap baggin function to make 3 samples.
    - for each sample, run a simple DT from question 1.
    - then average the accuracy. 
- Classification using RandomForestClassifier (5 pts)
- Evaluation (5 pts)

In [50]:
from random import randrange
df = pd.read_csv('./F21_CS559_HW3_data.csv')

def buildRandomForest(df):
    ratio = .5
    rand_i = []
    needed= round(len(df) * ratio)

    for i in range(needed):
        rand_i.append(randrange(len(df)))
    new_df = df.iloc[rand_i,:]
    return new_df

all_scores = []
for _ in range(3):
    rforest = buildRandomForest(df)
    r_score = DecisionTree(rforest)
    all_scores.append(r_score)
selfscore = sum(all_scores) / len(all_scores)

In [51]:
rfc = RandomForestClassifier()
rfc.fit(X,y)
score = rfc.score(X,y)
score

1.0

In [52]:
print(f"Random forest Implement score: {selfscore}")
print(f"sklearn score: {score}")

Random forest Implement score: 0.9632
sklearn score: 1.0


### Question 3: AdaBoost Classifier
- AB implementation (15 pts)
- Classification using AdaBoostClassifier (5 pts)
- Evaluation (5 pts)

In [16]:
# Compute error rate, alpha and w
def compute_error(y, y_pred, w_i):
    '''
    Calculate the error rate of a weak classifier m. Arguments:
    y: actual target value
    y_pred: predicted value by weak classifier
    w_i: individual weights for each observation
    
    Note that all arrays should be the same length
    '''
    return (sum(w_i * (np.not_equal(y, y_pred)).astype(int)))/sum(w_i)

def compute_alpha(error):
    '''
    Calculate the weight of a weak classifier m in the majority vote of the final classifier. This is called
    alpha in chapter 10.1 of The Elements of Statistical Learning. Arguments:
    error: error rate from weak classifier m
    '''
    return np.log((1 - error) / error)

def update_weights(w_i, alpha, y, y_pred):
    ''' 
    Update individual weights w_i after a boosting iteration. Arguments:
    w_i: individual weights for each observation
    y: actual target value
    y_pred: predicted value by weak classifier  
    alpha: weight of weak classifier used to estimate y_pred
    '''  
    return w_i * np.exp(alpha * (np.not_equal(y, y_pred)).astype(int))


# Define AdaBoost class
class AdaBoost:
    
    def __init__(self):
        self.alphas = []
        self.G_M = []
        self.M = None
        self.training_errors = []
        self.prediction_errors = []
        
    def predict(self, X):
        '''
        Predict using fitted model. Arguments:
        X: independent variables - array-like
        '''

        # Initialise dataframe with weak predictions for each observation
        weak_preds = pd.DataFrame(index = range(len(X)), columns = range(self.M)) 

        # Predict class label for each weak classifier, weighted by alpha_m
        for m in range(self.M):
            y_pred_m = self.G_M[m].predict(X) * self.alphas[m]
            weak_preds.iloc[:,m] = y_pred_m

        # Calculate final predictions
        y_pred = (1 * np.sign(weak_preds.T.sum())).astype(int)

        return y_pred

    def fit(self, X, y, M = 100):
        '''
        Fit model. Arguments:
        X: independent variables - array-like matrix
        y: target variable - array-like vector
        M: number of boosting rounds. Default is 100 - integer
        '''
        
        # Clear before calling
        self.alphas = [] 
        self.training_errors = []
        self.M = M

        # Iterate over M weak classifiers
        for m in range(0, M):
            
            # Set weights for current boosting iteration
            if m == 0:
                w_i = np.ones(len(y)) * 1 / len(y)  # At m = 0, weights are all the same and equal to 1 / N
            else:
                # (d) Update w_i
                w_i = update_weights(w_i, alpha_m, y, y_pred)
            
            # (a) Fit weak classifier and predict labels
            G_m = DecisionTreeClassifier(max_depth = 1)     # Stump: Two terminal-node classification tree
            G_m.fit(X, y, sample_weight = w_i)
            y_pred = G_m.predict(X)
            
            self.G_M.append(G_m) # Save to list of weak classifiers

            # (b) Compute error
            error_m = compute_error(y, y_pred, w_i)
            self.training_errors.append(error_m)

            # (c) Compute alpha
            alpha_m = compute_alpha(error_m)
            self.alphas.append(alpha_m)

        assert len(self.G_M) == len(self.alphas)

myabc= AdaBoost()
myabc.fit(X,y,500)
pred = myabc.predict(X) 
print(pred)
print(accuracy_score(pred,y))

0       1
1       1
2       1
3       1
4       1
       ..
2495    1
2496    1
2497    1
2498    1
2499    1
Length: 2500, dtype: int32
0.4996


In [ ]:
abc = AdaBoostClassifier()
abc.fit(X,y)
score = abc.score(X,y)
score

### Question 4: Gradient Boost Classifier
- GB implementation (15 pts)
- Classification using GradientBoostingClassifier (5 pts)
- Evaluation (5 pts)

In [56]:
import typing
from sklearn.linear_model import Ridge
def GradBoost(model,
              X_train: np.array,         # training independent vars
              y_train: np.array,         # training dependent var
              boosting_rounds: int = 100,# number of boosting rounds
              learning_rate: float = 0.1,# learning rate
              verbose: bool = True       # shows progress bar
              ) -> np.array: 
    '''
    Takes in a model and performs gradient boosting using it.
    '''
    import numpy as np
    
    # initalize guess of our training target variable using the mean
    y_hat_train = np.repeat(np.mean(y_train), len(y_train))
    
    # initialize out of sample prediction with training mean
    y_hat_train_test = np.repeat(np.mean(y_train), len(X_train))
    # calculate the training residuals fusing the first guess
    pseudo_resids = y_train - y_hat_train
    
    # performs gradient boosting with a tqdm progress bar
    if verbose:
        from tqdm import tqdm
        # iterates through the boosting round
        for _ in tqdm(range(0, boosting_rounds)):
            # fit the model to the pseudo residuals
            model = model.fit(X_train, pseudo_resids)
            # increment y_hat_train with the predicted resids*lr
            y_hat_train += learning_rate * model.predict(X_train)  
     
            # increment the predicted test y as well
            y_hat_train_test += (learning_rate *   
                                 model.predict(X_train))
            # calculate the pseudo resids for next round
            pseudo_resids = y_train - y_hat_train
    # performs gradient boosting without a progress bar        
    else:
        # iterates through the boosting round
        for _ in range(0, boosting_rounds):
            # fit the model to the pseudo residuals
            model = model.fit(X_train, pseudo_resids)
            # increment the y_hat_train with the pseudo resids*lr
            y_hat_train += learning_rate * model.predict(X_train)
       
            # increment the predicted test y as well
            y_hat_train_test += (learning_rate * 
                                 model.predict(X_train))
            # calculate the pseudo resids for next round
            pseudo_resids = y_train - y_hat_train
    # return a tuple of the training y_hat and the test y_hat
    return model.predict(X_train)

df = pd.read_csv('./F21_CS559_HW3_data.csv')
X = df.drop('class', 1)
y = df['class']
mygb = GradBoost(DecisionTreeClassifier(), X,y)
print(mygb)
#selfscore = accuracy_score(mygb, y)

  0%|          | 0/100 [00:00<?, ?it/s]


ValueError: Unknown label type: 'continuous'

In [19]:
gbc = GradientBoostingClassifier()
gbc.fit(X,y)
pred = gbc.predict(X)
print(accuracy_score(y,pred))

1.0
